In [34]:
import re
import string
import nltk
import regex as re 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
file_path = r"C:\Users\Sujal Karmakar\Desktop\Desktop\Data Analyst\Python\python_data_analytics_project\Theme Finder Using Caption (NLP)\Data\leanbeast_analysis_ready.csv"

df = pd.read_csv(file_path)

#### 1) Basic Cleaning (lower case, remover numbers, emoji, etc)

In [ ]:
df_caption = df[["content_id", "caption_text"]]

In [ ]:
import re

# list of generic words and hashtags that i wanna remove
generic_words = set(
    [
        "fitness",
        "gym",
        "workout",
        "fitnessmotivation",
        "bodybuilding",
        "training",
        "fit",
        "muscle",
        "fitfam",
        "gymlife",
        "shredded",
        "sixpack",
        "fitspo",
        "personaltrainer",
        "fitnessmodel",
        "exercise",
        "comment",
        "follow",
        "share",
        "message",
        "dm",
        "whatsapp",
        "send",
        "will",
        "i",
        "interested in paid transformation",
        "now",
        "amazing",
        "total",
        "this is just an act",
        "free diet plan",
        "zero",
        "link",
        "tap",
        "check",
        "list",
    ]
)


def clean_caption(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text_lower = text.lower()

    text_lower = re.sub(r"#(\w+)", r"\1", text_lower)

    text_lower = re.sub(r"\s+", " ", text_lower)

    for phrase in generic_words:
        # Add word boundaries \b to avoid partial matches
        pattern = r"\b" + re.escape(phrase) + r"\b"
        text_lower = re.sub(pattern, "", text_lower, flags=re.IGNORECASE)

    text_lower = re.sub(r"\s+", " ", text_lower).strip()

    return text_lower


df_caption_cleaned = df[["content_id", "caption_text"]].copy()

df_caption_cleaned["caption_text_cleaned"] = df_caption_cleaned["caption_text"].apply(
    clean_caption
)

print(df_caption_cleaned.head())

    content_id                                       caption_text  \
0  DJ0c4FlylXc  Well done @jai.bajaj0786 lost 39kgs without co...   
1  DF65FG3yT-Z  ⚠️⚠️‼️TRANSFORMATION ALERT ⚠️⚠️‼️  If he keen ...   
2  C8OUNStSKC7  Shredded mode on @manavkansagra1111 under my g...   
3  DKGgJptyiJ-  Comment “Atta” and I will send you the link of...   
4  DKFZihkSSQj  Dinner Date ❤️🧿❤️🥂  #couple #love #couplegoals...   

                                caption_text_cleaned  
0  well done @jai.bajaj0786 lost 39kgs without co...  
1  ⚠️⚠️‼️transformation alert ⚠️⚠️‼️ if he keen d...  
2  mode on @manavkansagra1111 under my guidance… ...  
3  “atta” and you the of healthy atta in your inbox.  
4  dinner date ❤️🧿❤️🥂 couple love couplegoals wed...  


In [51]:
df_caption_cleaned.sample(15)

,content_id,caption_text,caption_text_cleaned
341,DCB0JnByd6x,Comment “QNT” and i will send you original lin...,“qnt” and you original of protein delivered in...
957,C0GG2bTvrh2,Tsunami bar vs Traditional bar .. ??,tsunami bar vs traditional bar .. ??
340,DCEPhyYyz39,Do this instead✅,do this instead✅
234,DExdY7US4t6,Lowest body fat 7%,lowest body fat 7%
459,C_eqNLfSDbU,Cola is unhealthy but fruit juice is healthy??...,cola is unhealthy but fruit juice is healthy??...
1403,CkkOfQGpEBM,Kuch sikho inse… 😀 #abs #fitness #gym #workout...,kuch sikho inse… 😀 abs motivation gymmotivatio...
723,C5noQaBywNV,Sasta low carb zero sugar diet… VEG OPTION NA...,sasta low carb sugar diet… veg option navratri...
562,C86wZ8aylVC,5 lucky winners to win 2kg whey protein… Comment,5 lucky winners to win 2kg whey protein…
105,DH3AJ3WSu_Y,Comment “Cauli” and i will send you cauli rice...,“cauli” and you cauli rice recipe .
345,DB8CzLFS421,Comment “Insulin” and i will send you 5 food o...,“insulin” and you 5 food options help control ...


#### 2) Tokenization (removing stop words)

In [ ]:
# import nltk

# # Define a custom download directory
# custom_nltk_path = r'C:\Users\Sujal Karmakar\.conda\envs\DS\nltk_data'

# # Download resources to the custom path
## nltk.download('punkt', download_dir=custom_nltk_path)
# nltk.download('punkt_tab', download_dir=custom_nltk_path)
# nltk.download('stopwords', download_dir=custom_nltk_path)

# # Manually add the path to NLTK's search locations
# nltk.data.path.append(custom_nltk_path)

# i had to explicitly download these by specifying files and also punkt was not working for me nltk was demanding punkt_tab so i did that 


#### 3) Creating dictionaries and corpus for LDA (Linear discriminant analysis)

In [ ]:
from bertopic import BERTopic

documents = df_caption_cleaned["caption_text_cleaned"].tolist()

topic_model = BERTopic(language="english", nr_topics=25, verbose=True)

topics, probs = topic_model.fit_transform(documents)

topic_info = topic_model.get_topic_info()
print(topic_info)


for topic_num in topic_info["Topic"]:
    if topic_num != -1:  # -1 is the noise topic
        print(f"\nTopic {topic_num}:")
        print(topic_model.get_topic(topic_num))

2025-06-01 16:04:13,636 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/49 [00:00<?, ?it/s]

2025-06-01 16:04:23,473 - BERTopic - Embedding - Completed ✓
2025-06-01 16:04:23,473 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-01 16:04:25,776 - BERTopic - Dimensionality - Completed ✓
2025-06-01 16:04:25,777 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-01 16:04:25,823 - BERTopic - Cluster - Completed ✓
2025-06-01 16:04:25,823 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-01 16:04:25,887 - BERTopic - Representation - Completed ✓
2025-06-01 16:04:25,887 - BERTopic - Topic reduction - Reducing number of topics
2025-06-01 16:04:25,899 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-01 16:04:25,960 - BERTopic - Representation - Completed ✓
2025-06-01 16:04:25,962 - BERTopic - Topic reduction - Reduced number of topics from 31 to 25


    Topic  Count                                       Name  \
0      -1    536                         -1_body_you_and_to   
1       0    126  0_healthylifestyle_sport_crossfit_healthy   
2       1    125                  1_biceps_abs_cardio_gains   
3       2    101                          2_hai_nahi_meh_ka   
4       3     81                  3_protein_veg_whey_source   
5       4     66                    4_inbox_eggs_recipe_you   
6       5     59  5_wedding_photography_happy_photooftheday   
7       6     57                  6_natty_mykonos_wife_itna   
8       7     53          7_india_iamleanbeastindia_with_he   
9       8     43                       8_sugar_is_to_stevia   
10      9     38                   9_fatloss_to_meal_weight   
11     10     34                    10_ice_icebath_bath_the   
12     11     29                    11_keto_on_gains_biceps   
13     12     29              12_cholesterol_high_heart_the   
14     13     26           13_fasting_hours_autophagy_s

In [ ]:
with open("bertopic_topics_2.txt", "w", encoding="utf-8") as f:
    for idx, topic in topic_model.get_topic_info().iterrows():
        f.write(f"Topic {topic['Topic']} (Count: {topic['Count']}):\n")
        terms = topic_model.get_topic(topic["Topic"])
        for term, weight in terms:
            f.write(f"  {term}: {weight:.4f}\n")
        f.write("\n")

In [ ]:
# Mapping topic IDs to clear, professional names
topic_labels = {
    -1: "Nutrition & Diet Tips",
    0: "Fitness & Exercise",
    1: "Health Awareness",
    2: "Nutrition & Diet Tips",
    3: "Weight Management",
    4: "Nutrition & Diet Tips",
    5: "Lifestyle & Motivation",
    6: "Insights & Commentary ",
    7: "Lifestyle & Motivation",
    8: "Fitness & Exercise",
    9: "Sugar & Metabolic Health",
    10: "Weight Management",
    11: "Recovery & Injury Prevention",
    12: "Health Awareness",
    13: "Health Awareness",
    14: "Lifestyle & Motivation",
    15: "Nutrition & Diet Tips",
    16: "Health Awareness",
    17: "Health Awareness",
    18: "Nutrition & Diet Tips",
    19: "Health Awareness",
    20: "Lifestyle & Motivation",
    21: "Nutrition & Diet Tips",
    22: "Nutrition & Diet Tips",
    23: "Sugar & Metabolic Health",
}

In [ ]:
df_topics = pd.DataFrame(
    {"content_id": df_caption_cleaned["content_id"], "Topic_ID": topics}
)

df_topics["Topic_Name"] = df_topics["Topic_ID"].map(topic_labels)

df_final = df.merge(df_topics, on="content_id", how="left")

print(df_final.head())


df_final.to_csv("leanbeast_analysis_ready_captions_with_topics.csv", index=False)

             Username post_type   content_id day_of_week  hour_of_day  \
0  iamleanbeast_india      Post  DJ0c4FlylXc      Monday            8   
1  iamleanbeast_india      Post  DF65FG3yT-Z     Tuesday           10   
2  iamleanbeast_india      Post  C8OUNStSKC7    Saturday            9   
3  iamleanbeast_india      Reel  DKGgJptyiJ-      Monday            8   
4  iamleanbeast_india      Post  DKFZihkSSQj      Sunday           22   

  time_am_pm video_duration aspect_ratio  \
0   08:40 AM            NaN        01:01   
1   10:36 AM            NaN        01:01   
2   09:58 AM            NaN        01:01   
3   08:57 AM          00:59        09:16   
4   10:38 PM            NaN        04:05   

                                        caption_text  caption_length  ...  \
0  Well done @jai.bajaj0786 lost 39kgs without co...             112  ...   
1  ⚠️⚠️‼️TRANSFORMATION ALERT ⚠️⚠️‼️  If he keen ...             158  ...   
2  Shredded mode on @manavkansagra1111 under my g...             